# 03 - Predictability & Universe Filtering

1. Compute rolling predictability metrics for each ticker
2. Visualize and compare scores across universe and time
3. Select top-N most “learnable” tickers for RL agent
4. Document all decisions, assumptions, and open questions


In [1]:
# SETUP: Imports & Paths ===========================
import jupyter

import os
import pandas as pd


from tqdm import tqdm
from src.data.feature_pipeline import basic_chart_features,load_base_dataframe
from src.predictability.easiness import rolling_sharpe, rolling_r2, rolling_info_ratio, rolling_autocorr
from src.predictability.pipeline import generate_universe_easiness_report
from IPython import display


C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# LOAD OHLCV ==========================================
ohlcv_df = load_base_dataframe()
ohlcv_df.tail()

,id,symbol,timestamp,date,open,high,low,close,volume,trade_count,...,vwap_change,trade_count_change,sector_id,industry_id,return_1d,vix,vix_norm,sp500,sp500_norm,market_return_1d
429701,429702,SPY,2025-05-29 04:00:00,2025-05-29,593.06,593.2000,586.070,590.05,70073758.0,826143.0,...,-0.000250,0.128714,None,None,0.003947,0.1918,-0.006732,59.1217,0.004011,0.004011
429702,429703,SPY,2025-05-30 04:00:00,2025-05-30,588.93,591.1299,583.235,589.39,90601242.0,884337.0,...,-0.002564,0.070441,None,None,-0.001119,0.1857,-0.031804,59.1169,-0.000081,-0.000081
429703,429704,SPY,2025-06-02 04:00:00,2025-06-02,587.76,592.7900,585.060,592.71,61630502.0,728812.0,...,0.003093,-0.175866,None,None,0.005633,0.1836,-0.011309,59.3594,0.004102,0.004102
429704,429705,SPY,2025-06-03 04:00:00,2025-06-03,592.34,597.0800,591.850,596.09,63606204.0,690792.0,...,0.009093,-0.052167,None,None,0.005703,0.1769,-0.036492,59.7037,0.005800,0.005800
429705,429706,SPY,2025-06-04 04:00:00,2025-06-04,596.96,597.9500,595.490,595.93,57314199.0,629200.0,...,0.002856,-0.089161,None,None,-0.000268,0.1761,-0.004522,59.7081,0.000074,0.000074


In [3]:
# CROP THE SAMPLE =======================================
tickers = ['AAPL','GOOGL']

In [4]:

all_metric = generate_universe_easiness_report(ohlcv_df,tickers)

  0%|          | 0/2 [00:00<?, ?it/s]

[WARNING] Metric rolling_max_rdawdown not recognized, skipping.
[WARNING] Metric rolling_variacne_ratio not recognized, skipping.


C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Model is not converging.  Current: 150.87608610063188 is not greater than 150.9697835836491. Delta is -0.09369748301722325
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Model is not converging.  Current: 150.97490713622793 is not greater than 150.98894425874195. Delta is -0.014037122514025668
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is

C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\sit

C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Model is not converging.  Current: 135.35741982176646 is not greater than 135.39975492287525. Delta is -0.04233510110879024
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Model is not converging.  Current: 135.418636819212 is not greater than 135.46216071242432. Delta is -0.04352389321232408
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is k

...

C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\sit

C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Model is not converging.  Current: 157.8158557771731 is not greater than 157.8441406166131. Delta is -0.028284839439976395
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Model is not converging.  Current: 157.8082391710461 is not greater than 157.86783453765753. Delta is -0.05959536661143261
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is k

C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\sit

C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\sit

KeyError: 'close'

In [ ]:
all_metric.dropna()